In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm   #for文の進捗を確認

In [2]:
# ペルソナを意識して情報収集の条件を変更
# 目黒区、2K, 2DK, 2LDK

load_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13110&cb=0.0&ct=9999999&et=9999999&md=05&md=06&md=07&ts=1&ts=2&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2="

res = requests.get(load_url)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

In [3]:
# ページ数の取得
pages = soup.find("ol", class_="pagination-parts")
num_of_pages = int(pages.find_all("li")[-1].text)

In [4]:
# データフレーム作成
rental_property_datas = pd.DataFrame(columns=["名前", "住所", "最寄り駅１","最寄り駅２","最寄り駅３", "築年数", "建物全体の階数", "階数", "賃料", "管理費", "敷金", "礼金", "間取り", "面積" ])

In [5]:
for p in tqdm(range(num_of_pages)):
# for p in tqdm(range(2)):

    # topのページは"&page=1"を付けても開ける
    page_url = load_url + "&page=" + str(p+1)
    res = requests.get(page_url)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    
    # サーバー負荷を避けるため1s遅延
    time.sleep(1)

    # 「建物ごとに表示」ページから物件毎の情報を取得
    cassetitems = soup.find_all("div", class_="cassetteitem")

    for i in range(len(cassetitems)):
        
        # 上の部分（建物情報）
        details = cassetitems[i].find_all("div", class_="cassetteitem-detail")
        
        for ii in range(len(details)):
            name = details[ii].find("div", class_="cassetteitem_content-title").text
            address = details[ii].find("li", class_="cassetteitem_detail-col1").text
            _stations = details[ii].find_all("div", class_="cassetteitem_detail-text")
            station1 = _stations[0].text
            station2 = _stations[1].text
            station3 = _stations[2].text
            _col3 = details[ii].find_all("li", class_="cassetteitem_detail-col3")
            building_age = _col3[0].find_all("div")[0].text
            number_of_floors = _col3[0].find_all("div")[1].text
            
            # 下の部分（部屋の情報）
            items = cassetitems[i].find("div", class_="cassetteitem-item")
            tbodys = items.find_all("tbody")
            for iii in range(len(tbodys)):
                _tds = tbodys[iii].find_all("td")
                floor = _tds[2].text.replace('\r','').replace('\n','').replace('\t','')
                rent = _tds[3].find("span", class_="cassetteitem_other-emphasis ui-text--bold").text
                maintenance_fee = _tds[3].find("span", class_="cassetteitem_price cassetteitem_price--administration").text
                deposit = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--deposit").text
                gratuity = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--gratuity").text
                layout = _tds[5].find("span", class_="cassetteitem_madori").text
                area = _tds[5].find("span", class_="cassetteitem_menseki").text
                
                # DataFrameにまとめた後、rental_property_datasへ追加する
                _d = pd.DataFrame()
                _d["名前"] = [name]
                _d["住所"] = [address]
                _d["最寄り駅１"] = [station1]
                _d["最寄り駅２"] = [station2]
                _d["最寄り駅３"] = [station3]
                _d["築年数"] = [building_age]
                _d["建物全体の階数"] = [number_of_floors]
                _d["階数"] = [floor]
                _d["賃料"] = [rent]
                _d["管理費"] = [maintenance_fee]
                _d["敷金"] = [deposit]
                _d["礼金"] = [gratuity]
                _d["間取り"] = [layout]
                _d["面積"] = [area]

                rental_property_datas = pd.concat([rental_property_datas, _d], ignore_index=True)

                

  0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 22/22 [00:50<00:00,  2.31s/it]


In [6]:
# 重複判定(データをクレンジングせず重複判定してみる)

df = rental_property_datas
df_Final = df[df[["住所","賃料","管理費","間取り","階数","面積","敷金","礼金"]].duplicated()]
len(df_Final)

343

In [7]:
rental_property_datas.head()

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,築14年,3階建,1階,21万円,5000円,-,-,2DK,45.43m2
1,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,築14年,3階建,3階,22万円,5000円,22万円,-,2DK,45.43m2
2,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,築14年,3階建,3階,22.5万円,5000円,-,-,2DK,51.89m2
3,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,築14年,3階建,1-3階,22.5万円,5000円,-,-,2DK,51.89m2
4,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,築14年,3階建,1階,23万円,5000円,23万円,-,2DK,45.43m2


In [8]:
rental_property_datas.to_csv('SUUMO_meguro_raw.csv', index=False, encoding='utf-8-sig')  # 生データを眺めてデータの質を改善する

In [9]:
# 築年数
rental_property_datas["築年数"].unique()

array(['築14年', '築16年', '新築', '築29年', '築11年', '築24年', '築3年', '築51年', '築9年',
       '築25年', '築33年', '築42年', '築45年', '築31年', '築47年', '築35年', '築54年',
       '築2年', '築18年', '築23年', '築34年', '築1年', '築37年', '築5年', '築38年',
       '築22年', '築15年', '築7年', '築8年', '築4年', '築44年', '築30年', '築28年',
       '築43年', '築49年', '築17年', '築53年', '築40年', '築21年', '築52年', '築13年',
       '築56年', '築10年', '築46年', '築32年', '築48年', '築41年', '築50年', '築57年',
       '築6年', '築27年', '築12年', '築36年', '築0年', '築26年', '築20年', '築39年',
       '築55年', '築59年', '築65年', '築58年', '築19年'], dtype=object)

In [10]:
for i in range(len(rental_property_datas)):
    if rental_property_datas["築年数"][i] == "新築":
        rental_property_datas["築年数"][i] = "築0年"  # 築〇年に揃える
    rental_property_datas["築年数"][i] = int(rental_property_datas["築年数"][i].replace("築","").replace("年",""))  # 数字部分だけ残してintに変換


rental_property_datas["築年数"].unique()

array([14, 16, 0, 29, 11, 24, 3, 51, 9, 25, 33, 42, 45, 31, 47, 35, 54, 2,
       18, 23, 34, 1, 37, 5, 38, 22, 15, 7, 8, 4, 44, 30, 28, 43, 49, 17,
       53, 40, 21, 52, 13, 56, 10, 46, 32, 48, 41, 50, 57, 6, 27, 12, 36,
       26, 20, 39, 55, 59, 65, 58, 19], dtype=object)

In [11]:
#建物全体の階数
rental_property_datas["建物全体の階数"].unique()

array(['3階建', '6階建', '地下1地上2階建', '4階建', '地下1地上4階建', '地下1地上7階建',
       '地下1地上10階建', '5階建', '7階建', '2階建', '地下1地上3階建', '15階建', '10階建',
       '11階建', '地下1地上18階建', '14階建', '17階建', '地下2地上25階建', '地下1地上14階建',
       '25階建', '地下1地上5階建', '8階建', '12階建', '19階建', '地下1地上6階建', '地下2地上27階建',
       '18階建', '42階建', '地下2地上11階建', '13階建', '9階建', '地下1地上12階建',
       '地下3地上5階建', '地下1地上8階建', '16階建', '地下2地上12階建', '地下3地上3階建',
       '地下2地上18階建', '地下1地上15階建', '地下1地上11階建'], dtype=object)

In [12]:
# 最小階数と最大回数のコラムに直す
# 地下〇地上×　->　"最小_階数"=〇、"最大_階数"=×
# 〇階建て -> "最小_階数"=0, "最大_階数"=〇
_df_min_max = pd.DataFrame(columns=["建物階数_最小", "建物階数_最大"])
for i in range(len(rental_property_datas)):
    _df = pd.DataFrame()
    if "地下" in rental_property_datas["建物全体の階数"][i]:
        _df["建物階数_最小"] = [int(rental_property_datas["建物全体の階数"][i].split("地上")[0].replace("地下","-"))]
        _df["建物階数_最大"] = [int(rental_property_datas["建物全体の階数"][i].split("地上")[1].replace("階建",""))]
    else:
        _df["建物階数_最小"] = [int(0)]
        _df["建物階数_最大"] = [int(rental_property_datas["建物全体の階数"][i].replace("階建",""))]
     
    _df_min_max = pd.concat([_df_min_max, _df], axis=0, ignore_index=True)

rental_property_datas = pd.concat([rental_property_datas, _df_min_max], axis=1)
rental_property_datas

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積,建物階数_最小,建物階数_最大
0,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1階,21万円,5000円,-,-,2DK,45.43m2,0,3
1,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,3階,22万円,5000円,22万円,-,2DK,45.43m2,0,3
2,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,3階,22.5万円,5000円,-,-,2DK,51.89m2,0,3
3,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1-3階,22.5万円,5000円,-,-,2DK,51.89m2,0,3
4,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1階,23万円,5000円,23万円,-,2DK,45.43m2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,マンション雅叙苑,東京都目黒区下目黒１,ＪＲ山手線/目黒駅 歩8分,東急目黒線/不動前駅 歩8分,ＪＲ山手線/五反田駅 歩13分,54,15階建,14階,50万円,-,100万円,50万円,2LDK,125.81m2,0,15
1098,中目黒マンション,東京都目黒区上目黒１,東急東横線/中目黒駅 歩3分,東急東横線/代官山駅 歩7分,ＪＲ山手線/恵比寿駅 歩13分,59,地下1地上10階建,7階,35万円,-,-,35万円,2SLDK,74.89m2,-1,10
1099,日生住宅目黒マンション,東京都目黒区目黒３,ＪＲ山手線/目黒駅 歩13分,東急東横線/中目黒駅 歩15分,東急目黒線/不動前駅 歩16分,52,14階建,12階,22万円,-,22万円,22万円,2LDK,68m2,0,14
1100,ＪＲ山手線 目黒駅 地下1地上12階建 築50年,東京都目黒区下目黒４,ＪＲ山手線/目黒駅 歩15分,東急目黒線/不動前駅 歩19分,東急東横線/祐天寺駅 歩22分,50,地下1地上12階建,10階,18万円,-,18万円,18万円,2LDK,52.22m2,-1,12


In [13]:
# 階数
rental_property_datas["階数"].unique()

array(['1階', '3階', '1-3階', '2階', '4階', '5階', '6階', '7階', '10階', 'B1-1階',
       '13階', '8階', '15階', '1-2階', '2-3階', '11階', '3-4階', '17階', '18階',
       '16階', '14階', 'B1階', '22階', '21階', '4-5階', '5-6階', '9階', '12階',
       '30階', '37階'], dtype=object)

In [14]:
# 階数
# 階数が跨るモノは、階数_最小, 階数_最大に分ける
# 階数が単一のものは、階数_最小=階数_最大=階数にする

def replace_B_to_minus(floor: str) -> int:
    """地下を示すBを"-"に置き換える

    Args:
        floor (str): _description_

    Returns:
        int: _description_
    """
    if "B" in floor :
        floor = int(floor.replace("B","-"))
    else :
        floor = int(floor)

    return floor

_df_min_max = pd.DataFrame(columns=["階数_最小", "階数_最大"])
for i in range(len(rental_property_datas)):
    _df = pd.DataFrame()
    if "-" in rental_property_datas["階数"][i]:
        _df["階数_最小"] = [replace_B_to_minus(rental_property_datas["階数"][i].split("-")[0])]
        _df["階数_最大"] = [replace_B_to_minus(rental_property_datas["階数"][i].split("-")[1].replace("階", ""))]
    else:
        _df["階数_最小"] = [replace_B_to_minus(rental_property_datas["階数"][i].replace("階", ""))]
        _df["階数_最大"] = [replace_B_to_minus(rental_property_datas["階数"][i].replace("階", ""))]
     
    _df_min_max = pd.concat([_df_min_max, _df], axis=0, ignore_index=True)

rental_property_datas = pd.concat([rental_property_datas, _df_min_max], axis=1)
rental_property_datas

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積,建物階数_最小,建物階数_最大,階数_最小,階数_最大
0,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1階,21万円,5000円,-,-,2DK,45.43m2,0,3,1,1
1,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,3階,22万円,5000円,22万円,-,2DK,45.43m2,0,3,3,3
2,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,3階,22.5万円,5000円,-,-,2DK,51.89m2,0,3,3,3
3,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1-3階,22.5万円,5000円,-,-,2DK,51.89m2,0,3,1,3
4,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1階,23万円,5000円,23万円,-,2DK,45.43m2,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,マンション雅叙苑,東京都目黒区下目黒１,ＪＲ山手線/目黒駅 歩8分,東急目黒線/不動前駅 歩8分,ＪＲ山手線/五反田駅 歩13分,54,15階建,14階,50万円,-,100万円,50万円,2LDK,125.81m2,0,15,14,14
1098,中目黒マンション,東京都目黒区上目黒１,東急東横線/中目黒駅 歩3分,東急東横線/代官山駅 歩7分,ＪＲ山手線/恵比寿駅 歩13分,59,地下1地上10階建,7階,35万円,-,-,35万円,2SLDK,74.89m2,-1,10,7,7
1099,日生住宅目黒マンション,東京都目黒区目黒３,ＪＲ山手線/目黒駅 歩13分,東急東横線/中目黒駅 歩15分,東急目黒線/不動前駅 歩16分,52,14階建,12階,22万円,-,22万円,22万円,2LDK,68m2,0,14,12,12
1100,ＪＲ山手線 目黒駅 地下1地上12階建 築50年,東京都目黒区下目黒４,ＪＲ山手線/目黒駅 歩15分,東急目黒線/不動前駅 歩19分,東急東横線/祐天寺駅 歩22分,50,地下1地上12階建,10階,18万円,-,18万円,18万円,2LDK,52.22m2,-1,12,10,10


In [15]:
rental_property_datas["賃料"].unique()

array(['21万円', '22万円', '22.5万円', '23万円', '17.6万円', '18.3万円', '18.5万円',
       '18.8万円', '19万円', '19.1万円', '19.3万円', '19.5万円', '20万円', '20.1万円',
       '20.5万円', '20.6万円', '20.7万円', '20.8万円', '20.9万円', '23.5万円',
       '11.8万円', '12万円', '15.3万円', '25.4万円', '28.4万円', '30万円', '34.9万円',
       '16万円', '16.5万円', '16.7万円', '13.5万円', '26万円', '32万円', '27.4万円',
       '31万円', '18.6万円', '19.4万円', '19.6万円', '19.7万円', '19.8万円', '19.9万円',
       '20.4万円', '21.7万円', '13.8万円', '30.5万円', '38.5万円', '10.8万円', '11万円',
       '15.5万円', '9.5万円', '11.3万円', '11.5万円', '9.9万円', '7.9万円', '8万円',
       '8.2万円', '9万円', '21.9万円', '22.3万円', '17.2万円', '18万円', '18.7万円',
       '9.8万円', '43万円', '17.3万円', '18.1万円', '18.4万円', '12.7万円', '13万円',
       '12.3万円', '24.5万円', '26.5万円', '28.5万円', '29.5万円', '31.5万円',
       '37.5万円', '17.4万円', '14万円', '24万円', '27.5万円', '28.3万円', '24.6万円',
       '25万円', '32.9万円', '35.9万円', '39.6万円', '40万円', '46万円', '47万円',
       '22.7万円', '27.3万円', '10.5万円', '25.5万円', '30.9万円', '34万円', '44.7万円

In [16]:
# 賃料
for i in range(len(rental_property_datas)):
    if "." in rental_property_datas["賃料"][i] :
        rental_property_datas["賃料"][i] = int(rental_property_datas["賃料"][i].split(".")[0]) * 10000 +  int(float("0."+rental_property_datas["賃料"][i].split(".")[1].replace("万円", ""))*10000)
    else:
        rental_property_datas["賃料"][i] = int(rental_property_datas["賃料"][i].replace("万円", ""))* 10000

rental_property_datas["賃料"].unique()

array([210000, 220000, 225000, 230000, 176000, 183000, 185000, 188000,
       190000, 191000, 193000, 195000, 200000, 201000, 205000, 206000,
       207000, 208000, 209000, 235000, 118000, 120000, 153000, 254000,
       284000, 300000, 349000, 160000, 165000, 167000, 135000, 260000,
       320000, 274000, 310000, 186000, 194000, 196000, 197000, 198000,
       199000, 204000, 217000, 138000, 305000, 385000, 108000, 110000,
       155000, 95000, 113000, 115000, 99000, 79000, 80000, 82000, 90000,
       219000, 223000, 172000, 180000, 187000, 98000, 430000, 173000,
       181000, 184000, 127000, 130000, 123000, 245000, 265000, 285000,
       295000, 315000, 375000, 174000, 140000, 240000, 275000, 283000,
       246000, 250000, 329000, 359000, 396000, 400000, 460000, 470000,
       227000, 273000, 105000, 255000, 309000, 340000, 447000, 269000,
       228000, 233000, 258000, 243000, 279000, 75000, 350000, 351000,
       358000, 360000, 398000, 427000, 438000, 500000, 648000, 131000,
      

In [17]:
rental_property_datas["管理費"].unique()

array(['5000円', '-', '15000円', '3000円', '20000円', '12000円', '2000円',
       '3500円', '10000円', '7000円', '4000円', '6000円', '9000円', '30000円',
       '50000円', '8000円', '25000円', '1000円', '16000円', '11000円', '10500円',
       '17000円', '14000円', '29000円', '9500円', '6500円', '18000円', '15500円',
       '22000円', '13000円'], dtype=object)

In [18]:
for i in range(len(rental_property_datas)):
    if "-" in rental_property_datas["管理費"][i] :
        rental_property_datas["管理費"][i] = int(0)
    else:
        rental_property_datas["管理費"][i] = int(rental_property_datas["管理費"][i].replace("円", ""))

rental_property_datas["管理費"].unique()

array([5000, 0, 15000, 3000, 20000, 12000, 2000, 3500, 10000, 7000, 4000,
       6000, 9000, 30000, 50000, 8000, 25000, 1000, 16000, 11000, 10500,
       17000, 14000, 29000, 9500, 6500, 18000, 15500, 22000, 13000],
      dtype=object)

In [19]:
rental_property_datas["敷金"].unique()

array(['-', '22万円', '23万円', '17.6万円', '11.8万円', '12万円', '15.3万円',
       '25.4万円', '28.4万円', '30万円', '34.9万円', '16万円', '16.5万円', '16.7万円',
       '27万円', '26万円', '32万円', '18.6万円', '19.4万円', '19.6万円', '19.7万円',
       '19.8万円', '19.9万円', '20.1万円', '20.4万円', '20.9万円', '21.7万円',
       '13.8万円', '10.8万円', '11万円', '9.5万円', '11.3万円', '11.5万円', '9.9万円',
       '7.9万円', '8万円', '8.2万円', '9万円', '21万円', '9.8万円', '43万円', '17.3万円',
       '9.05万円', '18.4万円', '23.5万円', '12.7万円', '13万円', '12.3万円', '20.5万円',
       '22.5万円', '24.5万円', '26.5万円', '28.5万円', '29.5万円', '31.5万円',
       '37.5万円', '17.2万円', '17.4万円', '14万円', '24万円', '27.5万円', '30.5万円',
       '32.9万円', '35.9万円', '39.6万円', '40万円', '46万円', '47万円', '22.7万円',
       '20万円', '27.3万円', '25.5万円', '30.9万円', '34万円', '44.7万円', '26.9万円',
       '25.8万円', '23.3万円', '24.3万円', '35万円', '35.1万円', '35.8万円', '72万円',
       '36万円', '39.8万円', '42.7万円', '43.8万円', '50万円', '64.8万円', '13.1万円',
       '13.9万円', '28.6万円', '64.3万円', '16.9万円', '15万円', '26.2万円', '28.7万

In [20]:
for i in range(len(rental_property_datas)):
    if "-" in rental_property_datas["敷金"][i]:
        rental_property_datas["敷金"][i] = int(0)    
    elif "." in rental_property_datas["敷金"][i] :
        rental_property_datas["敷金"][i] = int(rental_property_datas["敷金"][i].split(".")[0]) * 10000 +  int(float("0."+rental_property_datas["敷金"][i].split(".")[1].replace("万円", ""))*10000)
    else:
        rental_property_datas["敷金"][i] = int(rental_property_datas["敷金"][i].replace("万円", ""))* 10000

rental_property_datas["敷金"].unique()

array([0, 220000, 230000, 176000, 118000, 120000, 153000, 254000, 284000,
       300000, 349000, 160000, 165000, 167000, 270000, 260000, 320000,
       186000, 194000, 196000, 197000, 198000, 199000, 201000, 204000,
       209000, 217000, 138000, 108000, 110000, 95000, 113000, 115000,
       99000, 79000, 80000, 82000, 90000, 210000, 98000, 430000, 173000,
       90500, 184000, 235000, 127000, 130000, 123000, 205000, 225000,
       245000, 265000, 285000, 295000, 315000, 375000, 172000, 174000,
       140000, 240000, 275000, 305000, 329000, 359000, 396000, 400000,
       460000, 470000, 227000, 200000, 273000, 255000, 309000, 340000,
       447000, 269000, 258000, 233000, 243000, 350000, 351000, 358000,
       720000, 360000, 398000, 427000, 438000, 500000, 648000, 131000,
       139000, 286000, 643000, 169000, 150000, 262000, 287000, 292000,
       155000, 100000, 298000, 410000, 355000, 385000, 395000, 226000,
       128000, 800000, 250000, 274000, 158000, 145000, 281000, 283000,
   

In [21]:
rental_property_datas["礼金"].unique()

array(['-', '17.6万円', '11.8万円', '12万円', '15.3万円', '16万円', '16.5万円',
       '16.7万円', '13.5万円', '18.6万円', '19.4万円', '19.6万円', '19.7万円',
       '19.8万円', '19.9万円', '20.1万円', '20.4万円', '20.9万円', '21.7万円',
       '13.8万円', '30.5万円', '38.5万円', '10.8万円', '11万円', '9.5万円', '11.3万円',
       '11.5万円', '9.9万円', '7.9万円', '8万円', '8.2万円', '9万円', '18万円', '9.8万円',
       '86万円', '34.6万円', '23万円', '27.15万円', '27.6万円', '23.5万円', '12.7万円',
       '13万円', '12.3万円', '22.5万円', '24.5万円', '26.5万円', '28.5万円', '29.5万円',
       '31.5万円', '37.5万円', '17.2万円', '17.4万円', '14万円', '32.9万円', '35.9万円',
       '39.6万円', '40万円', '43万円', '46万円', '47万円', '22.7万円', '20万円',
       '27.3万円', '10.5万円', '25.5万円', '26万円', '26.9万円', '25.8万円', '23.3万円',
       '24.3万円', '79.6万円', '85.4万円', '87.6万円', '100万円', '129.6万円',
       '13.1万円', '13.9万円', '28.6万円', '72万円', '128.6万円', '16.9万円', '15万円',
       '26.2万円', '28.7万円', '29.2万円', '27万円', '12.1万円', '15.5万円', '29.8万円',
       '41万円', '35.5万円', '79万円', '80万円', '22.6万円', '12.8万円', '27.4万

In [22]:
for i in range(len(rental_property_datas)):
    if "-" in rental_property_datas["礼金"][i]:
        rental_property_datas["礼金"][i] = int(0)    
    elif "." in rental_property_datas["礼金"][i] :
        rental_property_datas["礼金"][i] = int(rental_property_datas["礼金"][i].split(".")[0]) * 10000 +  int(float("0."+rental_property_datas["礼金"][i].split(".")[1].replace("万円", ""))*10000)
    else:
        rental_property_datas["礼金"][i] = int(rental_property_datas["礼金"][i].replace("万円", ""))* 10000

rental_property_datas["礼金"].unique()

array([0, 176000, 118000, 120000, 153000, 160000, 165000, 167000, 135000,
       186000, 194000, 196000, 197000, 198000, 199000, 201000, 204000,
       209000, 217000, 138000, 305000, 385000, 108000, 110000, 95000,
       113000, 115000, 99000, 79000, 80000, 82000, 90000, 180000, 98000,
       860000, 346000, 230000, 271500, 276000, 235000, 127000, 130000,
       123000, 225000, 245000, 265000, 285000, 295000, 315000, 375000,
       172000, 174000, 140000, 329000, 359000, 396000, 400000, 430000,
       460000, 470000, 227000, 200000, 273000, 105000, 255000, 260000,
       269000, 258000, 233000, 243000, 796000, 854000, 876000, 1000000,
       1296000, 131000, 139000, 286000, 720000, 1286000, 169000, 150000,
       262000, 287000, 292000, 270000, 121000, 155000, 298000, 410000,
       355000, 790000, 800000, 226000, 128000, 274000, 158000, 281000,
       291000, 109000, 220000, 125000, 65000, 370000, 478000, 78000,
       72000, 97000, 185000, 114000, 119000, 73000, 100000, 75000, 11700

In [23]:
rental_property_datas["間取り"].unique()  # 問題なし

array(['2DK', '2LDK', '2K', '2SLDK', '2SDK', '2SLK'], dtype=object)

In [24]:
rental_property_datas["面積"].unique()

array(['45.43m2', '51.89m2', '66.98m2', '40.06m2', '41.44m2', '40.14m2',
       '40.13m2', '40.31m2', '42.94m2', '40.07m2', '40.45m2', '41.8m2',
       '40.12m2', '53m2', '72.03m2', '68.39m2', '66.72m2', '40.62m2',
       '45m2', '45.63m2', '54.95m2', '44.22m2', '40.8m2', '40.84m2',
       '40.6m2', '38.31m2', '56.03m2', '30.85m2', '29.98m2', '31.01m2',
       '36.73m2', '35.72m2', '36m2', '32.18m2', '32.4m2', '30.4m2',
       '35.4m2', '33.12m2', '30m2', '28.72m2', '34.78m2', '31.47m2',
       '31.57m2', '43.52m2', '42.55m2', '40.04m2', '41.11m2', '40.16m2',
       '58.03m2', '35m2', '44.07m2', '48.02m2', '47.05m2', '91.55m2',
       '28m2', '54.84m2', '49.54m2', '56.74m2', '59.51m2', '40m2', '41m2',
       '45.83m2', '45.02m2', '41.76m2', '40.96m2', '40.17m2', '42.29m2',
       '49.99m2', '53.39m2', '63.57m2', '49.72m2', '35.64m2', '50m2',
       '49.35m2', '50.17m2', '54.56m2', '59.21m2', '40.92m2', '50.07m2',
       '57.24m2', '51.25m2', '50.84m2', '61.22m2', '77.36m2', '54.23m2',


In [25]:
for i in range(len(rental_property_datas)):
    if "." in rental_property_datas["面積"][i] :
        rental_property_datas["面積"][i] = float(rental_property_datas["面積"][i].split(".")[0])  +  float("0."+rental_property_datas["面積"][i].split(".")[1].replace("m2", ""))
    else :
        rental_property_datas["面積"][i] = float(rental_property_datas["面積"][i].replace("m2", ""))

rental_property_datas["面積"].unique()

array([45.43, 51.89, 66.98, 40.06, 41.44, 40.14, 40.13, 40.31, 42.94,
       40.07, 40.45, 41.8, 40.12, 53.0, 72.03, 68.39, 66.72, 40.62, 45.0,
       45.63, 54.95, 44.22, 40.8, 40.84, 40.6, 38.31, 56.03, 30.85, 29.98,
       31.01, 36.73, 35.72, 36.0, 32.18, 32.4, 30.4, 35.4, 33.12, 30.0,
       28.72, 34.78, 31.47, 31.57, 43.52, 42.55, 40.04, 41.11, 40.16,
       58.03, 35.0, 44.07, 48.02, 47.05, 91.55, 28.0, 54.84, 49.54, 56.74,
       59.51, 40.0, 41.0, 45.83, 45.02, 41.76, 40.96, 40.17, 42.29, 49.99,
       53.39, 63.57, 49.72, 35.64, 50.0, 49.35, 50.17, 54.56, 59.21,
       40.92, 50.07, 57.24, 51.25, 50.84, 61.22, 77.36, 54.23, 55.2,
       61.59, 58.1, 39.6, 77.01, 82.47, 47.06, 52.2, 51.46, 54.77, 53.57,
       59.59, 61.28, 66.91, 45.61, 55.11, 55.76, 51.06, 49.43, 34.55,
       64.5, 67.75, 64.34, 78.95, 68.83, 86.25, 105.83, 36.83, 41.6,
       55.47, 61.15, 76.82, 46.0, 65.16, 39.9, 74.15, 51.56, 42.36, 35.12,
       58.0, 37.06, 58.89, 51.8, 43.57, 44.09, 65.74, 72.25, 70

In [26]:
# クレンジング後
rental_property_datas

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積,建物階数_最小,建物階数_最大,階数_最小,階数_最大
0,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1階,210000,5000,0,0,2DK,45.43,0,3,1,1
1,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,3階,220000,5000,220000,0,2DK,45.43,0,3,3,3
2,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,3階,225000,5000,0,0,2DK,51.89,0,3,3,3
3,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1-3階,225000,5000,0,0,2DK,51.89,0,3,1,3
4,サクラノキテラス中目黒,東京都目黒区上目黒３,東急東横線/中目黒駅 歩6分,東急東横線/代官山駅 歩16分,東急東横線/祐天寺駅 歩12分,14,3階建,1階,230000,5000,230000,0,2DK,45.43,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,マンション雅叙苑,東京都目黒区下目黒１,ＪＲ山手線/目黒駅 歩8分,東急目黒線/不動前駅 歩8分,ＪＲ山手線/五反田駅 歩13分,54,15階建,14階,500000,0,1000000,500000,2LDK,125.81,0,15,14,14
1098,中目黒マンション,東京都目黒区上目黒１,東急東横線/中目黒駅 歩3分,東急東横線/代官山駅 歩7分,ＪＲ山手線/恵比寿駅 歩13分,59,地下1地上10階建,7階,350000,0,0,350000,2SLDK,74.89,-1,10,7,7
1099,日生住宅目黒マンション,東京都目黒区目黒３,ＪＲ山手線/目黒駅 歩13分,東急東横線/中目黒駅 歩15分,東急目黒線/不動前駅 歩16分,52,14階建,12階,220000,0,220000,220000,2LDK,68.0,0,14,12,12
1100,ＪＲ山手線 目黒駅 地下1地上12階建 築50年,東京都目黒区下目黒４,ＪＲ山手線/目黒駅 歩15分,東急目黒線/不動前駅 歩19分,東急東横線/祐天寺駅 歩22分,50,地下1地上12階建,10階,180000,0,180000,180000,2LDK,52.22,-1,12,10,10


In [27]:
# 重複判定

df = rental_property_datas
df_cleansing= df[df[["住所","賃料","管理費","間取り","建物階数_最小","建物階数_最大","階数_最小","階数_最大","面積","敷金","礼金"]].duplicated()]
len(df_cleansing)

305

In [28]:
df_cleansing

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積,建物階数_最小,建物階数_最大,階数_最小,階数_最大
31,柿の木坂Ｋ＆Ｆ,東京都目黒区柿の木坂２,東急東横線/都立大学駅 歩10分,東急東横線/学芸大学駅 歩16分,,29,地下1地上2階建,1階,120000,3000,120000,120000,2DK,41.8,-1,2,1,1
66,東急目黒線 武蔵小山駅 6階建 新築,東京都目黒区目黒本町３,東急目黒線/武蔵小山駅 歩5分,東急目黒線/西小山駅 歩13分,東急池上線/戸越銀座駅 歩22分,0,6階建,2階,185000,15000,0,0,2DK,40.14,0,6,2,2
67,東急目黒線 武蔵小山駅 6階建 新築,東京都目黒区目黒本町３,東急目黒線/武蔵小山駅 歩5分,東急目黒線/西小山駅 歩13分,東急池上線/戸越銀座駅 歩22分,0,6階建,2階,188000,15000,0,0,2DK,41.44,0,6,2,2
68,東急目黒線 武蔵小山駅 6階建 新築,東京都目黒区目黒本町３,東急目黒線/武蔵小山駅 歩5分,東急目黒線/西小山駅 歩13分,東急池上線/戸越銀座駅 歩22分,0,6階建,1階,190000,15000,0,0,2DK,40.14,0,6,1,1
69,東急目黒線 武蔵小山駅 6階建 新築,東京都目黒区目黒本町３,東急目黒線/武蔵小山駅 歩5分,東急目黒線/西小山駅 歩13分,東急池上線/戸越銀座駅 歩22分,0,6階建,3階,190000,15000,0,0,2DK,40.14,0,6,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,スウィフト東山,東京都目黒区東山３,東急田園都市線/池尻大橋駅 歩1分,京王井の頭線/駒場東大前駅 歩18分,京王井の頭線/神泉駅 歩17分,31,地下1地上5階建,4-5階,285000,15000,285000,285000,2LDK,76.23,-1,5,4,5
1091,東急東横線 中目黒駅 10階建 築57年,東京都目黒区青葉台３,東急東横線/中目黒駅 歩10分,東急田園都市線/池尻大橋駅 歩7分,京王井の頭線/神泉駅 歩14分,57,10階建,5階,160000,9000,320000,80000,2LDK,51.97,0,10,5,5
1093,東急東横線 学芸大学駅 4階建 築33年,東京都目黒区柿の木坂３,東急東横線/学芸大学駅 歩15分,東急東横線/都立大学駅 歩18分,東急田園都市線/三軒茶屋駅 歩26分,33,4階建,1階,230000,0,230000,0,2LDK,60.7,0,4,1,1
1094,目黒台スカイマンション,東京都目黒区三田２,ＪＲ山手線/恵比寿駅 歩11分,ＪＲ山手線/目黒駅 歩10分,東急東横線/代官山駅 歩19分,45,11階建,2階,250000,0,250000,250000,2LDK,61.9,0,11,2,2


In [94]:
# 重複判定

df = rental_property_datas
df_Final = df[df[["名前","住所","賃料","管理費","間取り","階数","面積","敷金","礼金"]].duplicated()]
len(df_Final)

60

In [28]:
# csvへ保存
df_Final.to_csv('SUUMO_meguro_2K2DK2LDK.csv', index=False, encoding='utf-8-sig')

In [10]:
# 重複判定（参考）
# suumoサイトだけから抽出している分には、そのまま比較すればよいはず
df = rental_property_datas

# 全項目で判定
print(len(df[df[["名前", "住所", "最寄り駅１","最寄り駅２","最寄り駅３", "築年数", "建物全体の階数", "階数", "賃料", "管理費", "敷金", "礼金", "間取り", "面積"]].duplicated()]))

# 建物の情報は同じ場所から抜き出すことになるので、名前と住所を確認すれば良さそう
# お金に関わる項目はすべて入れる必要がある
# 階数は当然として、間取りと面積も、同じ建物内の場所で違う可能性あるので入れないといけない
print(len(df[df[["名前","住所","賃料","管理費","間取り","階数","面積","敷金","礼金"]].duplicated()]))
print(len(df[df[["名前","住所","賃料","管理費","間取り","階数","面積"]].duplicated()]))
print(len(df[df[["名前","住所","賃料","管理費","間取り","階数"       ]].duplicated()]))
print(len(df[df[["名前","住所","賃料",        "間取り","階数","面積"]].duplicated()]))
print(len(df[df[["名前",       "賃料","管理費","間取り","階数","面積"]].duplicated()]))


40
45
48
66
60
50
